In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [22]:
from sklearn.preprocessing import RobustScaler,OneHotEncoder,OrdinalEncoder,LabelEncoder
from sklearn.impute import SimpleImputer
import os,sys

In [82]:
df  = pd.read_parquet('sample_bo.parquet.gzip')
df.sample(5000)

,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,sales_9_month,...,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop,went_on_backorder
1518751,135.0,52.0,0.0,0.0,0.0,0.0,0.0,6.0,9.0,12.0,...,0.0,0.99,0.99,0.0,Yes,No,Yes,Yes,No,No
832526,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.98,0.98,0.0,No,No,No,No,No,No
513496,5.0,4.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,6.0,...,0.0,0.73,0.78,0.0,No,No,No,Yes,No,No
1632768,134.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.18,0.33,0.0,No,No,No,Yes,No,No
622171,-1.0,8.0,0.0,3.0,4.0,5.0,1.0,4.0,4.0,4.0,...,0.0,0.57,0.74,1.0,No,No,Yes,Yes,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1386737,0.0,8.0,0.0,1168.0,1856.0,2176.0,0.0,0.0,0.0,0.0,...,0.0,0.97,0.98,0.0,Yes,No,Yes,No,No,No
1584024,8.0,2.0,0.0,0.0,0.0,0.0,1.0,1.0,4.0,6.0,...,0.0,0.82,0.84,0.0,No,No,No,Yes,No,No
710783,1.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.99,0.98,0.0,No,No,No,Yes,No,No
632679,30.0,8.0,0.0,0.0,0.0,0.0,2.0,3.0,3.0,4.0,...,0.0,0.80,0.82,0.0,No,No,No,Yes,No,No


In [83]:
df.head()


,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,sales_9_month,...,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop,went_on_backorder
732013,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.98,0.98,0.0,No,No,Yes,Yes,No,No
38200,16.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.70,0.75,0.0,No,No,No,Yes,No,No
1145713,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.92,0.96,0.0,Yes,No,Yes,Yes,No,No
1080013,3.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.62,0.62,0.0,Yes,No,No,Yes,No,No
1264726,8.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-99.00,-99.00,0.0,Yes,No,No,No,No,No


In [84]:
X=df.drop('went_on_backorder',axis=1)
y=df['went_on_backorder']

In [85]:
X_train,X_test,y_train,y_test = train_test_split(X,y)


In [86]:
numerical_features = [feature for feature in df.columns if df[feature].dtype !='O']
categorical_features = [feature for feature in df.columns if feature not in numerical_features]

In [87]:
tr1 = Pipeline([
            ('si',SimpleImputer(strategy='median')),
            ('rs',RobustScaler()),
        ])
tr2 = Pipeline([
        ('oh',OrdinalEncoder())
        ])

preprocessor = ColumnTransformer([
        ('num',tr1,numerical_features),
        ('cat',tr2,categorical_features[:-1])
],remainder='passthrough')

In [88]:
X_train_tf = pd.DataFrame(preprocessor.fit_transform(X_train),columns=df.columns[:-1])

In [89]:
X_train_tf

,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,sales_9_month,...,potential_issue,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop
0,1.644737,0.00,0.0,0.00,0.000000,0.00,0.75,0.733333,0.677419,0.543478,...,0.0,0.515152,0.620690,0.0,0.0,0.0,0.0,1.0,1.0,0.0
1,0.815789,1.00,0.0,0.00,0.666667,1.60,1.50,1.800000,2.741935,2.391304,...,0.0,-0.272727,-0.448276,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.855263,-1.50,0.0,10.00,6.666667,6.00,3.75,3.133333,3.419355,3.652174,...,0.0,-0.121212,0.068966,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,2.763158,-1.50,0.0,0.00,0.000000,0.00,1.00,1.066667,2.258065,1.826087,...,0.0,-0.030303,-0.034483,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,-0.171053,1.00,0.0,0.00,0.000000,0.00,0.00,-0.066667,-0.064516,-0.086957,...,0.0,-0.272727,-0.068966,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374995,0.065789,-1.50,0.0,0.00,0.000000,0.00,0.00,-0.066667,-0.064516,-0.086957,...,0.0,-0.424242,-0.586207,0.0,0.0,0.0,0.0,0.0,1.0,0.0
374996,3.052632,-1.50,0.0,0.00,0.000000,0.00,0.00,-0.066667,-0.064516,1.000000,...,0.0,-0.303030,-0.172414,0.0,0.0,0.0,0.0,0.0,1.0,0.0
374997,-0.157895,0.00,0.0,0.00,0.000000,0.05,0.00,0.000000,-0.032258,-0.043478,...,0.0,-1.969697,-1.655172,0.0,0.0,0.0,0.0,1.0,1.0,0.0
374998,-0.092105,-0.75,0.0,0.25,0.166667,0.10,0.00,-0.066667,-0.064516,-0.086957,...,0.0,-0.515152,-0.413793,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [90]:
X_test_tf = pd.DataFrame(preprocessor.transform(X_test),columns=df.columns[:-1])

In [91]:
le =LabelEncoder()

In [92]:
y_train_tf = le.fit_transform(y_train)

In [93]:
y_test_tf = le.transform(y_test)

In [94]:
from imblearn.combine import SMOTETomek
s = SMOTETomek(random_state=42,sampling_strategy='minority')
X_train,y_train = s.fit_resample(X_train_tf,y_train_tf)
X_test,y_test=s.fit_resample(X_test_tf,y_test_tf)

ValueError: Expected n_neighbors <= n_samples,  but n_samples = 1, n_neighbors = 6

In [95]:
sum(y_test_tf)

895

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf= RandomForestClassifier()

In [48]:
rf.fit(X_train_tf,y_train_tf)

RandomForestClassifier()

In [29]:
p =rf.predict(X_test)

In [35]:
from sklearn.metrics import f1_score
f1_score(y_test,p)

0.0

In [36]:
p

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0])

In [96]:
from backorder.utils import load_numpy_array

In [97]:
arr = load_numpy_array('/config/workspace/artifacts/07_02_23__20_58_48/data_transformation/transformed/train.npz')

In [99]:
import pandas as pd

In [100]:
pd.DataFrame(data=arr)

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,-0.206612,0.000000,0.000000,0.000000,0.000000,-0.015385,-0.100000,-0.142857,-0.140845,-0.121495,...,0.000000,-0.827586,-0.758621,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
1,1.487603,0.000000,136.000000,9.000000,9.000000,6.630769,3.300000,3.314286,3.591549,3.607477,...,0.000000,-0.827586,-0.862069,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
2,4.561983,-1.000000,0.000000,0.000000,0.000000,-0.015385,0.000000,0.485714,0.450704,0.551402,...,0.000000,-0.413793,-0.206897,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
3,-0.223140,-1.500000,0.000000,0.062500,0.050000,0.046154,-0.100000,-0.114286,-0.112676,-0.121495,...,0.000000,0.517241,0.551724,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
4,-0.206612,0.250000,0.000000,0.000000,0.000000,-0.015385,-0.100000,-0.114286,-0.126761,-0.130841,...,0.000000,0.482759,0.517241,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475983,0.026497,0.000000,0.000000,8.862116,5.492268,4.296114,2.543300,2.448306,2.182372,2.229983,...,19.814429,-0.784926,-0.886241,0.000000,0.0,0.0,0.938143,0.0,0.0,1.0
475984,-0.233367,0.000000,0.000000,9.546593,5.152527,3.863621,3.854242,2.090562,1.569113,1.785350,...,0.000000,0.475165,0.438348,0.847473,0.0,0.0,0.152527,0.0,0.0,1.0
475985,-0.231405,-1.500000,0.000000,0.687500,0.478288,0.319030,-0.100000,-0.142857,-0.140845,-0.140187,...,0.000000,-0.172414,-0.068966,0.000000,0.0,0.0,1.000000,0.0,0.0,1.0
475986,-0.043390,0.249923,13.501076,88.266216,49.369505,41.039263,27.775807,20.436186,17.817356,17.694344,...,97.497694,-0.034462,0.077615,10.249001,0.0,0.0,0.000000,0.0,0.0,1.0
